##### 版權 2024 Google LLC.


In [3]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用函式呼叫進行結構化數據提取


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/tutorials/structured_data_extraction"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />在 Google AI 上查看</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/generative-ai-docs/blob/main/site/zh/tutorials/structured_data_extraction.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/doggy8088/generative-ai-docs/blob/main/site/zh/tutorials/structured_data_extraction.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上查看原始碼</a>
  </td>
</table>


在這個教學中，你會使用 Gemini API 從故事中提取一個結構化資料的範例，包括角色、關係、事情和地點的清單。


## 設定


In [10]:
!pip install -U -q google-generativeai

In [11]:
import pathlib
import textwrap

import google.generativeai as genai
import google.ai.generativelanguage as glm


from IPython.display import display
from IPython.display import Markdown

from google.api_core import retry

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

一旦你取得了 API 金鑰，請將其傳遞到 SDK。你可以用兩種方式進行此事：

* 將金鑰放入 `GOOGLE_API_KEY` 環境變數中 (SDK 會自動從中撷取)。
* 將金鑰傳遞到 `genai.configure(api_key=...)`


In [ ]:
try:
    # Used to securely store your API key
    from google.colab import userdata

    # Or use `os.getenv('API_KEY')` to fetch an environment variable.
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
except ImportError:
    import os
    GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

genai.configure(api_key=GOOGLE_API_KEY)

genai.configure(
  api_key=GOOGLE_API_KEY,
  client_options={'api_endpoint':'autopush-generativelanguage.sandbox.googleapis.com'})

## 範例任務

對於這個教學課程，你會從自然語言故事中萃取實體。舉例來說，下面是一個由 Gemini 所寫的故事。


In [6]:
new_story = False

if new_story:
  model = genai.GenerativeModel(model_name='gemini-1.0-pro')

  response = model.generate_content("""
      Write a long story about a girl with magic backpack, her family, and at
      least one other charater. Make sure everyone has names. Don't forget to
      describe the contents of the backpack, and where everyone and everything
      starts and ends up.""", request_options={'retry': retry.Retry()})
  story = response.text
  print(response.candidates[0].citation_metadata)
else:
  story = """In the quaint town of Willow Creek, nestled amidst rolling hills and whispering willows, resided a young girl named Anya. As she stepped out of the creaky wooden door of her modest cottage, her heart skipped a beat with excitement and anticipation. Today was her first day of school, and she couldn't wait to show off her prized possession - a magical backpack.\n\nHanded down to her from her grandmother, the backpack was no ordinary satchel. Its soft, emerald-green fabric shimmered with an ethereal glow, and its leather straps held secrets that only Anya knew. Within its capacious interior lay an enchanted world, filled with wonders that would ignite her imagination and change her life forever.\n\nAnya's parents, kind-hearted Elise and wise-bearded Edward, bid her farewell with warm embraces. "Remember, my dear," whispered her mother, "use your magic wisely and for good." Her father added, "Always seek knowledge, and let the backpack be your trusted companion."\n\nWith a skip in her step, Anya set off towards the town's only schoolhouse. On her way, she passed her best friend, Samuel, a curious and adventurous boy with a mischievous grin. "Hey, Anya," he called out. "Can I see your backpack?"\n\nAnya hesitated for a moment before unzipping the flap and revealing its contents. Samuel's eyes widened in amazement as he peered inside. There, nestled amidst pencils and notebooks, were a shimmering sword, a book of ancient spells, a tiny compass that always pointed north, and a magical key that could open any lock.\n\nTogether, they marveled at the backpack's wonders, promising to keep its secrets safe. As they approached the schoolhouse, Anya noticed a group of older children huddled together, their faces etched with fear. Curiosity getting the better of her, she cautiously approached.\n\n"What's wrong?" she asked.\n\nA tall, lanky boy stepped forward. "There's a monster in the forest," he stammered. "It's been terrorizing the town, attacking animals and even people."\n\nAnya's heart sank. The town of Willow Creek was small and peaceful, and the thought of a monster brought a shiver down her spine. She knew she had to do something to protect her family and friends.\n\nWithout a moment's hesitation, Anya opened her backpack and retrieved the shimmering sword. With a determined gleam in her eye, she turned to her terrified peers. "Don't worry," she said, her voice steady. "I'll take care of it."\n\nWith Samuel close behind her, Anya ventured into the shadowy depths of the forest. The trees seemed to whisper secrets as she passed, and the undergrowth rustled with unseen creatures. As they walked deeper into the forest, the air grew heavy and the ground beneath their feet trembled.\n\nSuddenly, they came to a clearing, and there before their eyes was the monster - a massive beast with sharp teeth, glowing red eyes, and claws that could crush a human with ease. The creature roared, a thunderous sound that shook the forest to its core.\n\nFear surged through Anya, but she refused to let it consume her. She drew the sword from its sheath and charged towards the monster. The blade shimmered in the sunlight, and as it struck the beast's hide, a blinding light erupted, enveloping everything in its radiance.\n\nWhen the light faded, the monster was gone, and in its place was a pile of shattered crystals. Anya had defeated the creature with the magic of her backpack, proving that even the smallest of objects could hold the greatest of powers.\n\nAs she and Samuel returned to the town, they were greeted as heroes. The people of Willow Creek rejoiced, and the legend of Anya, the girl with the magic backpack, was passed down through generations. And so, Anya continued her adventures, using the backpack's wonders to make the world a better place, one magical step at a time."""

In [9]:
to_markdown(story)

> In the quaint town of Willow Creek, nestled amidst rolling hills and whispering willows, resided a young girl named Anya. As she stepped out of the creaky wooden door of her modest cottage, her heart skipped a beat with excitement and anticipation. Today was her first day of school, and she couldn't wait to show off her prized possession - a magical backpack.
> 
> Handed down to her from her grandmother, the backpack was no ordinary satchel. Its soft, emerald-green fabric shimmered with an ethereal glow, and its leather straps held secrets that only Anya knew. Within its capacious interior lay an enchanted world, filled with wonders that would ignite her imagination and change her life forever.
> 
> Anya's parents, kind-hearted Elise and wise-bearded Edward, bid her farewell with warm embraces. "Remember, my dear," whispered her mother, "use your magic wisely and for good." Her father added, "Always seek knowledge, and let the backpack be your trusted companion."
> 
> With a skip in her step, Anya set off towards the town's only schoolhouse. On her way, she passed her best friend, Samuel, a curious and adventurous boy with a mischievous grin. "Hey, Anya," he called out. "Can I see your backpack?"
> 
> Anya hesitated for a moment before unzipping the flap and revealing its contents. Samuel's eyes widened in amazement as he peered inside. There, nestled amidst pencils and notebooks, were a shimmering sword, a book of ancient spells, a tiny compass that always pointed north, and a magical key that could open any lock.
> 
> Together, they marveled at the backpack's wonders, promising to keep its secrets safe. As they approached the schoolhouse, Anya noticed a group of older children huddled together, their faces etched with fear. Curiosity getting the better of her, she cautiously approached.
> 
> "What's wrong?" she asked.
> 
> A tall, lanky boy stepped forward. "There's a monster in the forest," he stammered. "It's been terrorizing the town, attacking animals and even people."
> 
> Anya's heart sank. The town of Willow Creek was small and peaceful, and the thought of a monster brought a shiver down her spine. She knew she had to do something to protect her family and friends.
> 
> Without a moment's hesitation, Anya opened her backpack and retrieved the shimmering sword. With a determined gleam in her eye, she turned to her terrified peers. "Don't worry," she said, her voice steady. "I'll take care of it."
> 
> With Samuel close behind her, Anya ventured into the shadowy depths of the forest. The trees seemed to whisper secrets as she passed, and the undergrowth rustled with unseen creatures. As they walked deeper into the forest, the air grew heavy and the ground beneath their feet trembled.
> 
> Suddenly, they came to a clearing, and there before their eyes was the monster - a massive beast with sharp teeth, glowing red eyes, and claws that could crush a human with ease. The creature roared, a thunderous sound that shook the forest to its core.
> 
> Fear surged through Anya, but she refused to let it consume her. She drew the sword from its sheath and charged towards the monster. The blade shimmered in the sunlight, and as it struck the beast's hide, a blinding light erupted, enveloping everything in its radiance.
> 
> When the light faded, the monster was gone, and in its place was a pile of shattered crystals. Anya had defeated the creature with the magic of her backpack, proving that even the smallest of objects could hold the greatest of powers.
> 
> As she and Samuel returned to the town, they were greeted as heroes. The people of Willow Creek rejoiced, and the legend of Anya, the girl with the magic backpack, was passed down through generations. And so, Anya continued her adventures, using the backpack's wonders to make the world a better place, one magical step at a time.

## 使用自然語言

大語言模型是一種強大的多任務工具。通常只要告訴 Gemini 你想要什麼，它就能順利進行。

Gemini API 沒有 JSON 模式，因此透過這種方式產生資料結構時，有幾件事需要注意：

- 有時解析會失敗。
- 無法嚴格強制執行架構。

你將在下一節中解決這些問題。首先，使用純文字寫出的架構來試試一個簡單的自然語言提示。這尚未最佳化：


In [ ]:
model = model = model = genai.GenerativeModel(
    model_name='gemini-1.0-pro')

response = model.generate_content(textwrap.dedent("""\
    Please return JSON describing the the people, places, things and relationships from this story using the following schema:

    {"people": list[PERSON], "places":list[PLACE], "things":list[THING], "relationships": list[RELATIONSHIP]}

    PERSON = {"name": str, "description": str, "start_place_name": str, "end_place_name": str}
    PLACE = {"name": str, "description": str}
    THING = {"name": str, "description": str, "start_place_name": str, "end_place_name": str}
    RELATIONSHIP = {"person_1_name": str, "person_2_name": str, "relationship": str}

    All fields are required.

    Important: Only return a single piece of valid JSON text.

    Here is the story:

    """) + story)


In [ ]:
response.text

'{"people": [{"name": "Anya", "description": "A young girl who possesses a magical backpack.", "start_place_name": "Willow Creek", "end_place_name": null}, {"name": "Elise", "description": "Anya\'s kind-hearted mother.", "start_place_name": "Willow Creek", "end_place_name": null}, {"name": "Edward", "description": "Anya\'s wise-bearded father.", "start_place_name": "Willow Creek", "end_place_name": null}, {"name": "Samuel", "description": "Anya\'s curious and adventurous best friend.", "start_place_name": "Willow Creek", "end_place_name": null}], "places": [{"name": "Willow Creek", "description": "A quaint town nestled amidst rolling hills and whispering willows."}, {"name": "The forest", "description": "A shadowy place with whispering trees and unseen creatures."}], "things": [{"name": "Magical backpack", "description": "A magical backpack that holds an enchanted world filled with wonders.", "start_place_name": "Anya\'s grandmother\'s house", "end_place_name": "Willow Creek"}, {"name"

那回傳了一個 json 字串。可以試著解析看看：


In [ ]:
import json

json_text = response.text.strip('`\r\n ').removeprefix('json')
print(json.dumps(json.loads(json_text), indent=4))

{
    "people": [
        {
            "name": "Anya",
            "description": "A young girl who possesses a magical backpack.",
            "start_place_name": "Willow Creek",
            "end_place_name": null
        },
        {
            "name": "Elise",
            "description": "Anya's kind-hearted mother.",
            "start_place_name": "Willow Creek",
            "end_place_name": null
        },
        {
            "name": "Edward",
            "description": "Anya's wise-bearded father.",
            "start_place_name": "Willow Creek",
            "end_place_name": null
        },
        {
            "name": "Samuel",
            "description": "Anya's curious and adventurous best friend.",
            "start_place_name": "Willow Creek",
            "end_place_name": null
        }
    ],
    "places": [
        {
            "name": "Willow Creek",
            "description": "A quaint town nestled amidst rolling hills and whispering willows."
        },
       

這相對簡單且常常有效，但你可能透過使用 API 的功能呼叫功能定義架構使之更嚴格/健全。


## 使用函式呼叫


如果你尚未完成[函式呼叫基礎](https://ai.google.dev/tutorials/function_calling_python_quickstart) 教學課程，務必先完成該課程。

透過函式呼叫函式及其參數會以 `glm.FunctionDeclaration` 來描述給 API。在基本情況下，SDK 可以從函式及其註解建立 `FunctionDeclaration`。SDK 目前無法處理巢狀 `OBJECT` (`dict`) 參數的描述。因此，現階段你需要明確定義它們。


### 定義模式

從定義 `person` 為具有字串欄位 `name`, `description`, `start_place_name`, `end_place_name` 的物件開始。


In [189]:
person = glm.Schema(
    type = glm.Type.OBJECT,
    properties = {
        'name':  glm.Schema(type=glm.Type.STRING),
        'description':  glm.Schema(type=glm.Type.STRING),
        'start_place_name': glm.Schema(type=glm.Type.STRING),
        'end_place_name': glm.Schema(type=glm.Type.STRING)
    },
    required=['name', 'description', 'start_place_name', 'end_place_name']
)

然後定義 people 為 `ARRAY` 類別的 `person` 物件：


In [190]:
people = glm.Schema(
    type=glm.Type.ARRAY,
    items=person
)

然後對每個你試圖提取實體做同樣的事情：


In [191]:
place = glm.Schema(
    type = glm.Type.OBJECT,
    properties = {
        'name':  glm.Schema(type=glm.Type.STRING),
        'description':  glm.Schema(type=glm.Type.STRING),
    }
)

places = glm.Schema(
    type=glm.Type.ARRAY,
    items=place
)

In [192]:
thing = glm.Schema(
  type = glm.Type.OBJECT,
  properties = {
      'name':  glm.Schema(type=glm.Type.STRING),
      'description':  glm.Schema(type=glm.Type.STRING),
  }
)

things = glm.Schema(
    type=glm.Type.ARRAY,
    items=thing
)

In [193]:
relationship = glm.Schema(
    type = glm.Type.OBJECT,
    properties = {
        'person_1_name':  glm.Schema(type=glm.Type.STRING),
        'person_2_name':  glm.Schema(type=glm.Type.STRING),
        'relationship':  glm.Schema(type=glm.Type.STRING),
    }
)

relationships = glm.Schema(
    type=glm.Type.ARRAY,
    items=relationship
)

現在建立 `FunctionDeclaration`：


In [194]:
add_to_database = glm.FunctionDeclaration(
    name="add_to_database",
    description=textwrap.dedent("""\
        Adds entities to the database.
        """),
    parameters=glm.Schema(
        type=glm.Type.OBJECT,
        properties = {
            'people': people,
            'places': places,
            'things': things,
            'relationships': relationships
        }
    )
)

### 呼叫 API

如同你在 [函式呼叫基本知識](https://ai.google.dev/tutorials/function_calling_python_quickstart) 中看到的，現在你可以將 `FunctionDeclaration` 傳遞給 `genai.GenerativeModel` 建構函式的 `tools` 參數，而 Construcotr 也會接受函式宣告的等效 JSON 表示法：


In [195]:
model = model = genai.GenerativeModel(
    model_name='gemini-1.0-pro',
    tools = [add_to_database])

每次你呼叫 API 時，SDK 將會將工具與你的提示訊息寄出，而模型應該呼叫你定義的那個功能：


In [196]:
result = model.generate_content(f"""
Please add the people, places, things, and relationships from this story to the database:

{story}
""")

現在沒有可剖析的文字，結果_就是_一個資料結構。


In [197]:
'text' in result.candidates[0].content.parts[0]

False

In [198]:
'function_call' in result.candidates[0].content.parts[0]

True

In [199]:
fc = result.candidates[0].content.parts[0].function_call
print(type(fc))

<class 'google.ai.generativelanguage_v1beta.types.content.FunctionCall'>


`glm.FunctionCall` 類別基於 Google 協定緩衝建立，將它轉成更熟悉的 JSON 相容物件：


In [200]:
print(json.dumps(type(fc).to_dict(fc), indent=4))

{
    "name": "add_to_database",
    "args": {
        "relationships": [
            {
                "relationship": "mother-daughter",
                "person_2_name": "Elise",
                "person_1_name": "Anya"
            },
            {
                "person_1_name": "Anya",
                "relationship": "father-daughter",
                "person_2_name": "Edward"
            },
            {
                "relationship": "best friends",
                "person_1_name": "Anya",
                "person_2_name": "Samuel"
            }
        ],
        "places": [
            {
                "name": "Willow Creek",
                "description": "a quaint town nestled amidst rolling hills and whispering willows"
            },
            {
                "name": "forest",
                "description": "a shadowy place with rustling undergrowth"
            }
        ],
        "things": [
            {
                "description": "a backpack with a shimmering 

## 結論

雖然 API 可以處理輸入純文字並輸出文字的結構化資料擷取問題，但使用函式呼叫可能會更加可靠，因為它允許定義嚴格的架構，並消除潛在容易出錯的分析步驟。
